In [23]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


First load the data and process it before training. Data should be in the previous folder from the one where the notebook is executed.

In [24]:
subset_1 = np.load("../data/subset_1.npy")
subset_2 = np.load("../data/subset_2.npy")
subset_3 = np.load("../data/subset_3.npy")

IMG_NUM = len(subset_1) + len(subset_2) + len(subset_3) #1196

#Combine all subsets and reshape to image dimensions
data = np.vstack((subset_1,subset_2,subset_3)).reshape(IMG_NUM,150,225,3)

#Normalize to (0,1)
normalized = data.astype('float32') / 255.0

#Split into train and test
train, test = train_test_split(normalized, test_size=0.2, random_state=42)
